In [1]:
"""
    IMPORTING LIBS
"""
import dgl

import numpy as np
import os
import time
import random

import torch
from notebook_utils import Args

Using backend: pytorch


In [2]:
#   Set manual seed

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.random.manual_seed(seed)
    dgl.random.seed(seed)
    if torch.cuda.is_available():
       torch.cuda.manual_seed(seed)
       torch.cuda.manual_seed_all(seed)
       torch.backends.cudnn.deterministic = True
       torch.backends.cudnn.benchmark = False
        
"""
    VIEWING MODEL CONFIG AND PARAMS
"""
def view_model_param(MODEL_NAME, net_params):
    model = gnn_model(MODEL_NAME, net_params)
    model = model.float()
    total_param = 0
    print("MODEL DETAILS:\n")
    print(model)
    for param in model.parameters():
        print(param.data.size())
        total_param += np.prod(list(param.data.size()))
    print('MODEL/Total parameters:', MODEL_NAME, total_param)
    return total_param

In [3]:
"""
    IMPORTING CUSTOM MODULES/METHODS
"""

from data.data import load_data # import dataset
from utils import get_configs, get_arguments, add_dir_name # import arguments and configurations
from nets.molecules_graph_regression.load_net import gnn_model # import all GNNS

RDKit WARNING: [17:56:50] Enabling RDKit 2019.09.3 jupyter extensions


In [4]:
def main(args):    
    """
        USER CONTROLS
    """
    #args = get_arguments()

    args, config, params, net_params = get_configs(args)

    #   define which pipeline to be used
    if params['swa'] == True:
        from pipeline_swa import train_val_pipeline_classification
    elif params['swag'] == True:
        from pipeline_swag import train_val_pipeline_classification
    elif (params['sgld'] == True) or (params['psgld'] == True):
        from pipeline_sgld import train_val_pipeline_classification
    else:
        if params['bbp'] == True:
            from nets.molecules_graph_regression.load_bbp_net import gnn_model # import all GNNS
        from pipeline import train_val_pipeline_classification


    DATASET_NAME = config['dataset']
    MODEL_NAME = config['model']

    # setting seeds
    set_seed(params['seed'])
    print("Seed Number of Models: "+str(params['seed']))
    print("Data Seed Number: "+str(params['data_seed']))


    dataset = load_data(DATASET_NAME, args.num_train, args.num_val, args.num_test, args.data_seed, params)

    # network parameters

    #   add task information for net_params loss
    net_params['task'] = 'classification'
    net_params['num_classes'] = dataset.num_classes
    net_params['num_atom_type'] = dataset.num_atom_type
    net_params['num_bond_type'] = dataset.num_bond_type
    out_dir = config['out_dir']

    root_ckpt_dir = out_dir + 'checkpoints/' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" +\
        str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    write_file_name = out_dir + 'results/result_' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" +\
        str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')
    root_output_dir = out_dir + 'outputs/outputs_' + MODEL_NAME + "_" + DATASET_NAME + "_GPU" +\
        str(config['gpu']['id']) + "_" + time.strftime('%Hh%Mm%Ss_on_%b_%d_%Y')

    dirs = root_ckpt_dir, write_file_name, root_output_dir
    dirs = add_dir_name(dirs, MODEL_NAME, config, params, net_params)

    net_params['total_param'] = view_model_param(MODEL_NAME, net_params)
    train_val_pipeline_classification(MODEL_NAME, DATASET_NAME, dataset, config, params, net_params, dirs)

In [5]:
if __name__ == "__main__":
    
    config = 'configs/gin_bace.json'
    
    args = Args(config)
    
    args.gpu_id = 0
    main(args)


cuda available with GPU: TITAN Xp
True
Seed Number of Models: 41
Data Seed Number: 222
Seed Number of Data: 222
train/valid scaffold split:  False
train/test scaffold split:  False
valid/test scaffold split:  True
preparing 1200 graphs for the TRAIN set...
preparing 150 graphs for the VALIDATION set...
preparing 150 graphs for the TEST set...
Time taken: 18.3424s
MODEL DETAILS:

GINNet(
  (ginlayers): ModuleList(
    (0): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
          (linear_1): Linear(in_features=128, out_features=128, bias=False)
          (linear_2): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
    (1): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
          (linear_1): Linear(in_features=128, out_features=128, bias=False)
          (linear_2): Linear(in_features=128, out_features=128, bias=False)
        )
      )
    )
    (2): GINLayer(
      (apply_func): ApplyNodeFunc(
        (mlp): MLP(
         

Epoch 0:   0%|          | 0/200 [00:00<?, ?it/s]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 1:   0%|          | 1/200 [00:00<02:15,  1.47it/s, lr=0.001, test_AUC=0.652, test_EC

Epoch 7:   4%|▎         | 7/200 [00:02<01:20,  2.41it/s, lr=0.001, test_AUC=0.596, test_ECE=0.07, time=0.398, train_AUC=0.617, train_ECE=0.0359, train_loss=0.665, val_AUC=0.708, val_ECE=0.0914, val_loss=0.708]  /home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43:

Epoch 14:   7%|▋         | 14/200 [00:05<01:13,  2.54it/s, lr=0.001, test_AUC=0.817, test_ECE=0.111, time=0.42, train_AUC=0.724, train_ECE=0.035, train_loss=0.61, val_AUC=0.791, val_ECE=0.12, val_loss=0.791]    /home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 15:   8%|▊         | 15/200 [00:05<01:12,  2.57it/s, lr=0.001, test_AUC=0.801, test_ECE=0.163, time=0.377, train_AUC=0.757, train_ECE=0.0478, train_loss=0.582, val_AUC=0.783, val_ECE=0.105, val_loss=0.783]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/

Epoch 147:  74%|███████▎  | 147/200 [00:55<00:20,  2.61it/s, lr=0.0001, test_AUC=0.911, test_ECE=0.118, time=0.368, train_AUC=0.98, train_ECE=0.0306, train_loss=0.176, val_AUC=0.83, val_ECE=0.175, val_loss=0.83]  /home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/envs/benchmark_gnn/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/paul/mol_reliable_gnn/train/metrics.py:43: RuntimeWarning: Mean of empty slice.
  acc_mean = label[bin_idx].mean()
Epoch 149:  74%|███████▍  | 149/200 [00:56<00:19,  2.63it/s, lr=0.0001, test_AUC=0.909, test_ECE=0.125, time=0.409, train_AUC=0.981, train_ECE=0.0296, train_loss=0.189, val_AUC=0.831, val_ECE=0.168, val_loss=0.831]/home/paul/mol_reliable_gnn/train/metrics.py:36: RuntimeWarning: Mean of empty slice.
  conf_mean = pred[bin_idx].mean()
/home/paul/anaconda3/

Test AUC: 0.9086
Test ECE: 0.1438
Train AUC: 0.9842
Train ECE: 0.0335
TOTAL TIME TAKEN: 82.5249s
AVG TIME PER EPOCH: 0.3822s
